# Extract source SBML-qual model

In [1]:
import os
import libsbml
import pandas as pd
import numpy as np
from collections import Counter
import re

biodivine_model_dir = '/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/'

In [2]:
# Extract source files from BioDivine
import os
import shutil

source_directory = biodivine_model_dir + "sources"
destination_directory = biodivine_model_dir

# Ensure the destination directory exists
os.makedirs(destination_directory, exist_ok=True)
# Iterate through the subfolders in the source directory
for subfolder in os.listdir(source_directory):
    subfolder_path = os.path.join(source_directory, subfolder)
    # Check if it's a directory
    if os.path.isdir(subfolder_path):
        # Path to the 'source.sbml' file in the current subfolder
        sbml_file_path = os.path.join(subfolder_path, "source.sbml")
        # Check if the file exists
        if os.path.isfile(sbml_file_path):
            # Define the target path in the destination folder
            target_path = os.path.join(destination_directory, f"{subfolder}_source.sbml")
            # Copy the file to the destination folder
            shutil.copy(sbml_file_path, target_path)
            # print(f"Copied: {sbml_file_path} to {target_path}")
        else:
            print(f"'source.sbml' not found in: {subfolder_path}")

'source.sbml' not found in: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/sources/140_BREAST-CANCER-DRUG-RESISTANCE
'source.sbml' not found in: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/sources/233_BREAST-CANCER-SIGNALLING-PATHWAYS-T47D-COMPLETE
'source.sbml' not found in: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/sources/264_PREIMPLANTATION-EMBRYO-TROPHECTODERM-1
'source.sbml' not found in: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/sources/084_ABA-INDUCED-STOMATAL-CLOSURE
'source.sbml' not found in: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/sources/237_COLORECTAL-TUMORIGENESIS-REVERSION-SWITCH
'source.sbml' not found in: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/sources/265_PREIMPLANTATION-EMBRYO-TROPHECTODERM-1-MATURE
'source.sbml' not found in: /Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/sources/075_INFLAMMATORY-BOWEL-DISEASE
'source.sbml' not found in: /Users/luna/Deskto

# Annotation statistics

In [15]:
RDF_TAG_ITEM = ['rdf:RDF',
                'xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"',
                'xmlns:dcterms="http://purl.org/dc/terms/"',
                'xmlns:vcard4="http://www.w3.org/2006/vcard/ns#"',
                'xmlns:bqbiol="http://biomodels.net/biology-qualifiers/"',
                'xmlns:bqmodel="http://biomodels.net/model-qualifiers/"']
RDF_TAG = ' '.join(RDF_TAG_ITEM)

def get_gene_annotation_string(sbml_file):
    """
    Extract GeneProducts annotatino string from an SBML-qual model.

    Parameters:
    - sbml_file (str): Path to the SBML file.

    Returns:
    - list: List of GeneProduct annotation strings.
    """
    # Read the SBML model
    reader = libsbml.SBMLReader()
    document = reader.readSBML(sbml_file)
    model = document.getModel()
    if model is None:
        print(f"Failed to read the model: {sbml_file}")
        return []
    # Check if the model uses the qual package
    qual_plugin = model.getPlugin("qual")
    if not qual_plugin:
        print(f"The model does not use the qual package: {sbml_file}")
        return []
    # Get GeneProducts from the plugin
    gene_annotations = []
    species = qual_plugin.getListOfQualitativeSpecies()
    for i in range(len(species)):
        one_species = species[i]
        annotation_string = one_species.getAnnotationString()
        gene_annotations.append(annotation_string)
    return gene_annotations

def extract_ontology_from_items(items_list):
    """
    Extract ontology from items and return a flat list of tuples.
    Each tuple contains (ontology type, ontology id).

    Parameters
    ----------
    items_list : list
        A list of string items containing ontology annotations.

    Returns
    -------
    list of tuples
        A flat list of (ontology type, ontology id).
    """
    result_identifiers = []
    for item in items_list:
        # Extract identifiers from "urn:miriam" URIs (urn:miriam:TYPE:ID)
        miriam_matches = re.findall(r'urn:miriam:([^"]+)"', item)
        for r in miriam_matches:
            if ':' in r:
                ontology_type, ontology_id = r.split(":", 1)
                result_identifiers.append((ontology_type, ontology_id))
        # Extract identifiers from identifiers.org URLs (identifiers.org/TYPE/ID)
        idorg_matches = re.findall(r'identifiers\.org/([^"/]+)/([^"/]+)', item)
        for ontology_type, ontology_id in idorg_matches:
            result_identifiers.append((ontology_type, ontology_id))
    return result_identifiers

def divideExistingAnnotation(inp_str, qualifier):
    """
    Parse the annotation string to extract items in multiple <rdf:Bag> elements 
    under the specified qualifier (e.g., <bqbiol:is>), and keep the rest in the container.
    Ensures namespaces and structure are preserved when creating an empty qualifier block.

    Parameters
    ----------
    inp_str: str
        The full annotation string.
    qualifier: str
        The qualifier to target (e.g., bqbiol:is, bqbiol:isDescribedBy).

    Returns
    -------
    dict:
        - 'container': The annotation string with the qualifier blocks replaced by a single empty block.
        - 'items': A list of <rdf:li> elements found in all <rdf:Bag> containers of the specified qualifier.
        - If no qualifier blocks are found, returns None.
    """

    # Regex to match all blocks for the specified qualifier, including attributes
    qualifier_pattern = rf"(<{qualifier}\b[^>]*?>\s*<rdf:Bag>.*?</rdf:Bag>\s*</{qualifier}>)"
    qualifier_matches = re.findall(qualifier_pattern, inp_str, re.DOTALL)

    if not qualifier_matches:
        return None  # Return None if no blocks for the qualifier are found

    # Collect all <rdf:li> elements from each matched block
    rdf_li_pattern = r"<rdf:li[^>]*\/>"
    items = []
    for block in qualifier_matches:
        items.extend(re.findall(rdf_li_pattern, block))

    # Extract the opening tag with attributes from the first match
    match_prefix = re.match(rf"<{qualifier}.*?>", qualifier_matches[0])
    if match_prefix:
        qualifier_opening = match_prefix.group()  # Capture opening tag with attributes
    else:
        qualifier_opening = f"<{qualifier}>"

    # Construct a single empty qualifier block using the preserved attributes
    empty_qualifier_block = (
        f"      {qualifier_opening}\n"
        f"        <rdf:Bag>\n"
        f"        </rdf:Bag>\n"
        f"      </{qualifier}>"
    )
    # Remove all original qualifier blocks from the container
    stripped_annotation = re.sub(qualifier_pattern, "", inp_str, flags=re.DOTALL).strip()

    # Reinsert the empty qualifier block with namespaces added
    if RDF_TAG not in stripped_annotation:
        container = stripped_annotation.replace(
            "<rdf:RDF",
            f"<{RDF_TAG}",
        )
    else:
        container = stripped_annotation

    container = container.replace(
        "</rdf:Description>",
        f"{empty_qualifier_block}\n    </rdf:Description>"
    )
    # Remove repeated blank lines
    container = re.sub(r"\n\s*\n", "\n", container)
    return {"container": container, "items": items}

def getOntologyFromString(string_annotation, description=False):
    """
    Parse string and return string annotation,
    marked as <bqbiol:is> or <bqbiol:isVersionOf>;
    (and extract the description using <bqbiol:isDescribedBy>)
    If neither exists, return None.

    Parameters
    ----------
    string_annotation: str
    description: bool
        If True, also extract the description using <bqbiol:isDescribedBy>.

    Returns
    -------
    list-tuple (ontology type, ontology id)
         Return [] if none is provided.
    """

    # Define the bqbiol qualifiers to search
    bqbiol_qualifiers = ['is', 'isVersionOf']
    if description:
        bqbiol_qualifiers.append('isDescribedBy')

    result_identifiers = []
    for one_qualifier in bqbiol_qualifiers:
        res = divideExistingAnnotation(string_annotation, 'bqbiol:' + one_qualifier)
        if res is None:
            continue
        items = res['items']
        res = extract_ontology_from_items(items)
        # remove empty lists and duplicates
        res = [r for r in res if r is not None and r != [] and r not in result_identifiers]
        result_identifiers.extend(res)

    return result_identifiers
def collect_ontology_statistics(model_dir, overall_output_csv, model_ontology_csv, description = False):
    """
    Collect ontology statistics of SBML models in a folder.

    Parameters:
    - folder_path (str): Path to the folder containing SBML models.
    - overall_output_csv (str): Path to save overall statistics as a CSV.
    - model_ontology_csv (str): Path to save per-model statistics as a CSV.
    - description (bool): If True, extract the description using <bqbiol:isDescribedBy>
    """
    overall_counter = Counter()
    model_data = []

    # Iterate over all SBML files in the folder
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.xml') or f.endswith('.sbml')]
    for idx, file_name in enumerate(model_files):
        print(f"We are at {idx}: Processing {file_name}")
        file_path = os.path.join(model_dir, file_name)
        gene_annotations = get_gene_annotation_string(file_path)

        model_counter = Counter()
        for gene_annotation in gene_annotations:
            resources = getOntologyFromString(gene_annotation, description = description)
            # Count ontology types
            if resources is not None and resources != []:
                for ontology_type, ontology_id in resources:
                    overall_counter[ontology_type] += 1
                    model_counter[ontology_type] += 1

        # Store model-level data
        for ontology_type, count in model_counter.items():
            model_data.append({
                'model': file_name,
                'ontology_type': ontology_type,
                'count': count
            })

    # Convert overall statistics to a DataFrame
    overall_stats_df = pd.DataFrame.from_dict(overall_counter, orient='index', columns=['count'])
    overall_stats_df.reset_index(inplace=True)
    overall_stats_df.columns = ['ontology_type', 'count']

    # Convert model-specific data to a DataFrame
    model_stats_df = pd.DataFrame(model_data)

    # Save the dataframes to CSV
    overall_stats_df.to_csv(overall_output_csv, index=False)
    model_stats_df.to_csv(model_ontology_csv, index=False)
    print(f"Overall statistics saved to {overall_output_csv}")
    print(f"Model-specific statistics saved to {model_ontology_csv}")

In [16]:
output_csv = biodivine_model_dir+"biodivine_260125_ontology_statistics.csv"
model_ontology_csv = biodivine_model_dir+"biodivine_260125_model_ontology_statistics.csv"
collect_ontology_statistics(biodivine_model_dir, output_csv, model_ontology_csv, description = True)

We are at 0: Processing 054_PC12-CELL-DIFFERENTIATION_source.sbml
We are at 1: Processing 038_SKBR3-BREAST-CELL-LINE-LONG-TERM_source.sbml
We are at 2: Processing 073_LYMPHOID-AND-MYELOID-CELL-SPECIFICATION_source.sbml
We are at 3: Processing 126_ORF3A_source.sbml
We are at 4: Processing 043_BORDETELLA-BRONCHISEPTICA_source.sbml
We are at 5: Processing 239_M1-SYNOVIAL-MACROPHAGE_source.sbml
We are at 6: Processing 161_DIFFERENTIATION-OF-MONOCYTES_source.sbml
We are at 7: Processing 078_IMMUNE-SYSTEM_source.sbml
We are at 8: Processing 009_YEAST-APOPTOSIS_source.sbml
We are at 9: Processing 072_LYMPHOPOIESIS-REGULATORY-NETWORK_source.sbml
We are at 10: Processing 065_TUMOUR-CELL-INVASION-AND-MIGRATION_source.sbml
We are at 11: Processing 198_PAIR-RULE-MODULE_source.sbml
We are at 12: Processing 090_MAPK-REDUCED-2_source.sbml
We are at 13: Processing 179_MICROENVIRONMENT-CONTROL_source.sbml
We are at 14: Processing 029_TOLL-PATHWAY-OF-DROSOPHILA_source.sbml
We are at 15: Processing 010_C

In [18]:
model_ontology = pd.read_csv(biodivine_model_dir+"biodivine_260125_model_ontology_statistics.csv")
print(f"Total number of models with annotation: {model_ontology['model'].nunique()}")
model_ontology[model_ontology['ontology_type'] == 'pubmed']

Total number of models with annotation: 86


,model,ontology_type,count
0,038_SKBR3-BREAST-CELL-LINE-LONG-TERM_source.sbml,pubmed,71
3,126_ORF3A_source.sbml,pubmed,2
5,043_BORDETELLA-BRONCHISEPTICA_source.sbml,pubmed,42
6,239_M1-SYNOVIAL-MACROPHAGE_source.sbml,pubmed,588
18,078_IMMUNE-SYSTEM_source.sbml,pubmed,31
...,...,...,...
288,033_BT474-BREAST-CELL-LINE-LONG-TERM_source.sbml,pubmed,72
289,030_SPZ-NETWORK-OF-DROSOPHILA_source.sbml,pubmed,46
290,024_BUDDING-YEAST-CELL-CYCLE_source.sbml,pubmed,57
291,027_WG-PATHWAY-OF-DROSOPHILA_source.sbml,pubmed,61


In [13]:
sbml = "/Users/luna/Desktop/CRBM/AMAS_proj/Models/BioDivine_260125/111_APOPTOSIS_source.sbml"
reader = libsbml.SBMLReader()
document = reader.readSBML(sbml)
model = document.getModel()
qual_spec = model.getPlugin("qual").getQualitativeSpecies('csa11')
print(qual_spec.getAnnotationString())

<annotation>
  <rdf:RDF>
    <rdf:Description rdf:about="#s_id_sa83">
      <bqbiol:isDescribedBy>
        <rdf:Bag>
          <rdf:li rdf:resource="urn:miriam:hgnc.symbol:BCL2"/>
        </rdf:Bag>
      </bqbiol:isDescribedBy>
      <bqbiol:isDescribedBy>
        <rdf:Bag>
          <rdf:li rdf:resource="urn:miriam:hgnc.symbol:BCL2"/>
        </rdf:Bag>
      </bqbiol:isDescribedBy>
      <bqbiol:isDescribedBy>
        <rdf:Bag>
          <rdf:li rdf:resource="urn:miriam:refseq:NM_000657"/>
        </rdf:Bag>
      </bqbiol:isDescribedBy>
      <bqbiol:isDescribedBy>
        <rdf:Bag>
          <rdf:li rdf:resource="urn:miriam:ncbigene:596"/>
        </rdf:Bag>
      </bqbiol:isDescribedBy>
      <bqbiol:isDescribedBy>
        <rdf:Bag>
          <rdf:li rdf:resource="urn:miriam:ncbigene:596"/>
        </rdf:Bag>
      </bqbiol:isDescribedBy>
      <bqbiol:isDescribedBy>
        <rdf:Bag>
          <rdf:li rdf:resource="urn:miriam:uniprot:P10415"/>
        </rdf:Bag>
      </bqbiol:i

## Run Converter

Test TabularQual converter by converting SBML → Spreadsheet → SBML and comparing counts.


In [22]:
import os
import sys
import libsbml
import pandas as pd
import re
from collections import Counter
import warnings

# Import TabularQual converter
sys.path.insert(0, '/Users/luna/Desktop/CRBM/Standards/TabularQual')
from TabularQual_converter.convert_sbml_to_spreadsheet import convert_sbml_to_spreadsheet
from TabularQual_converter.convert_spreadsheet_to_sbml import convert_spreadsheet_to_sbml
from TabularQual_converter.sbml_reader import read_sbml

# Create output directories
spreadsheet_output_dir = os.path.join(biodivine_model_dir, "spreadsheets")
sbml_output_dir = os.path.join(biodivine_model_dir, "converted_sbml")
os.makedirs(spreadsheet_output_dir, exist_ok=True)
os.makedirs(sbml_output_dir, exist_ok=True)

In [25]:
def count_sbml_stats(sbml_path):
    """
    Count species, transitions, and annotations from an SBML-qual file.
    
    Returns:
        dict with keys: species_count, transitions_count, 
            species_annotations_count, transition_annotations_count
    """
    try:
        reader = libsbml.SBMLReader()
        document = reader.readSBML(sbml_path)
        model = document.getModel()
        if model is None:
            return {"error": "No model found"}
        
        qual_plugin = model.getPlugin("qual")
        if not qual_plugin:
            return {"error": "No qual plugin found"}
        
        # Count species and their annotations
        species_count = qual_plugin.getNumQualitativeSpecies()
        species_annotations = 0
        for i in range(species_count):
            qs = qual_plugin.getQualitativeSpecies(i)
            if qs.isSetAnnotation():
                anno_str = qs.getAnnotationString()
                # Count number of rdf:li elements (each is an annotation)
                species_annotations += len(re.findall(r'<rdf:li[^>]*resource=', anno_str))
        
        # Count transitions and their annotations
        transitions_count = qual_plugin.getNumTransitions()
        transition_annotations = 0
        for i in range(transitions_count):
            tr = qual_plugin.getTransition(i)
            if tr.isSetAnnotation():
                anno_str = tr.getAnnotationString()
                transition_annotations += len(re.findall(r'<rdf:li[^>]*resource=', anno_str))
        
        return {
            "species_count": species_count,
            "transitions_count": transitions_count,
            "species_annotations": species_annotations,
            "transition_annotations": transition_annotations,
            "error": None
        }
    except Exception as e:
        return {"error": str(e)}


def count_spreadsheet_stats(xlsx_path):
    """
    Count species, transitions, and annotations from a spreadsheet.
    Uses TabularQual's read_spreadsheet_to_model function.
    
    Returns:
        dict with keys: species_count, transitions_count, 
            species_annotations_count, transition_annotations_count
    """
    try:
        from TabularQual_converter.spreadsheet_reader import read_spreadsheet_to_model
        
        model, _ = read_spreadsheet_to_model(xlsx_path)
        
        # Count species and their annotations
        species_count = len(model.species)
        species_annotations = sum(len(sp.annotations) for sp in model.species.values())
        
        # Count transitions and their annotations
        transitions_count = len(model.transitions)
        transition_annotations = sum(len(tr.annotations) for tr in model.transitions)
        
        return {
            "species_count": species_count,
            "transitions_count": transitions_count,
            "species_annotations": species_annotations,
            "transition_annotations": transition_annotations,
            "error": None
        }
    except Exception as e:
        return {"error": str(e)}


In [26]:
import time

results = []

# Get all SBML files
model_files = [f for f in os.listdir(biodivine_model_dir) 
               if (f.endswith('.xml') or f.endswith('.sbml')) and os.path.isfile(os.path.join(biodivine_model_dir, f))]
print(f"Found {len(model_files)} SBML models to process")

for idx, file_name in enumerate(model_files):
    print(f"\n[{idx+1}/{len(model_files)}] Processing: {file_name}")
    
    original_sbml_path = os.path.join(biodivine_model_dir, file_name)
    base_name = os.path.splitext(file_name)[0]
    
    # Output paths
    spreadsheet_path = os.path.join(spreadsheet_output_dir, f"{base_name}.xlsx")
    converted_sbml_path = os.path.join(sbml_output_dir, f"{base_name}_converted.sbml")
    
    result = {
        "model": file_name,
        "original_sbml_species": None,
        "original_sbml_transitions": None,
        "original_sbml_species_anno": None,
        "original_sbml_trans_anno": None,
        "spreadsheet_species": None,
        "spreadsheet_transitions": None,
        "spreadsheet_species_anno": None,
        "spreadsheet_trans_anno": None,
        "converted_sbml_species": None,
        "converted_sbml_transitions": None,
        "converted_sbml_species_anno": None,
        "converted_sbml_trans_anno": None,
        "sbml_to_xlsx_time": None,
        "xlsx_to_sbml_time": None,
        "sbml_to_xlsx_error": None,
        "xlsx_to_sbml_error": None,
    }
    
    # Count stats from original SBML
    orig_stats = count_sbml_stats(original_sbml_path)
    if orig_stats.get("error"):
        result["sbml_to_xlsx_error"] = orig_stats["error"]
        results.append(result)
        print(f"  Error reading original SBML: {orig_stats['error']}")
        continue
    
    result["original_sbml_species"] = orig_stats["species_count"]
    result["original_sbml_transitions"] = orig_stats["transitions_count"]
    result["original_sbml_species_anno"] = orig_stats["species_annotations"]
    result["original_sbml_trans_anno"] = orig_stats["transition_annotations"]
    
    # Convert SBML to spreadsheet
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            start_time = time.time()
            convert_sbml_to_spreadsheet(
                original_sbml_path, 
                spreadsheet_path, 
                template_path=None,  # No template to skip README/Appendix
                rule_format="operators"
            )
            sbml_to_xlsx_time = time.time() - start_time
            result["sbml_to_xlsx_time"] = round(sbml_to_xlsx_time, 3)
        print(f"  Converted to spreadsheet in {sbml_to_xlsx_time:.3f}s")
    except Exception as e:
        result["sbml_to_xlsx_error"] = str(e)
        results.append(result)
        print(f"  Error converting to spreadsheet: {e}")
        continue
    
    # Count stats from spreadsheet
    xlsx_stats = count_spreadsheet_stats(spreadsheet_path)
    if xlsx_stats.get("error"):
        result["sbml_to_xlsx_error"] = xlsx_stats["error"]
    else:
        result["spreadsheet_species"] = xlsx_stats["species_count"]
        result["spreadsheet_transitions"] = xlsx_stats["transitions_count"]
        result["spreadsheet_species_anno"] = xlsx_stats["species_annotations"]
        result["spreadsheet_trans_anno"] = xlsx_stats["transition_annotations"]
    
    # Convert spreadsheet back to SBML
    try:
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            start_time = time.time()
            convert_spreadsheet_to_sbml(
                spreadsheet_path,
                converted_sbml_path,
                interactions_anno=False, # no existing interaction annotations
                transitions_anno=True
            )
            xlsx_to_sbml_time = time.time() - start_time
            result["xlsx_to_sbml_time"] = round(xlsx_to_sbml_time, 3)
        print(f"  Converted back to SBML in {xlsx_to_sbml_time:.3f}s")
    except Exception as e:
        result["xlsx_to_sbml_error"] = str(e)
        results.append(result)
        print(f"  Error converting to SBML: {e}")
        continue
    
    # Count stats from converted SBML
    conv_stats = count_sbml_stats(converted_sbml_path)
    if conv_stats.get("error"):
        result["xlsx_to_sbml_error"] = conv_stats["error"]
    else:
        result["converted_sbml_species"] = conv_stats["species_count"]
        result["converted_sbml_transitions"] = conv_stats["transitions_count"]
        result["converted_sbml_species_anno"] = conv_stats["species_annotations"]
        result["converted_sbml_trans_anno"] = conv_stats["transition_annotations"]
    
    results.append(result)

print(f"\n\nProcessed {len(results)} models")


Found 205 SBML models to process

[1/205] Processing: 054_PC12-CELL-DIFFERENTIATION_source.sbml
  Error converting to spreadsheet: SBML parsing errors: A <layout> object must have the required attribute 'layout:id' and may have the optional attribute 'layout:name'. No other attributes from the SBML Level 3 Layout namespace are permitted on a <layout> object.
Reference: L3V1 Layout V1 Section 3.6
 Layout attribute 'id' is missing.


[2/205] Processing: 038_SKBR3-BREAST-CELL-LINE-LONG-TERM_source.sbml
Found 25 species
Found 21 transitions
Found 81 interactions
  Converted to spreadsheet in 0.063s
Found 25 species
Found 21 transitions
Found 81 interactions
  Converted back to SBML in 0.099s

[3/205] Processing: 073_LYMPHOID-AND-MYELOID-CELL-SPECIFICATION_source.sbml
Found 34 species
Found 32 transitions
Found 96 interactions
  Converted to spreadsheet in 0.066s
Found 34 species
Found 32 transitions
Found 96 interactions
  Converted back to SBML in 0.109s

[4/205] Processing: 126_ORF3A_sou

In [27]:
# Create results DataFrame and save to CSV
results_df = pd.DataFrame(results)

# Add comparison columns to check if counts match
results_df["species_match"] = (
    (results_df["original_sbml_species"] == results_df["spreadsheet_species"]) & 
    (results_df["spreadsheet_species"] == results_df["converted_sbml_species"])
)
results_df["transitions_match"] = (
    (results_df["original_sbml_transitions"] == results_df["spreadsheet_transitions"]) & 
    (results_df["spreadsheet_transitions"] == results_df["converted_sbml_transitions"])
)
# For annotations, we compare original to spreadsheet (converted may differ due to URI format)
results_df["species_anno_match"] = (
    results_df["original_sbml_species_anno"] == results_df["spreadsheet_species_anno"]
)
results_df["trans_anno_match"] = (
    results_df["original_sbml_trans_anno"] == results_df["spreadsheet_trans_anno"]
)

# Reorder columns for better readability
column_order = [
    "model",
    "original_sbml_species", "spreadsheet_species", "converted_sbml_species", "species_match",
    "original_sbml_transitions", "spreadsheet_transitions", "converted_sbml_transitions", "transitions_match",
    "original_sbml_species_anno", "spreadsheet_species_anno", "converted_sbml_species_anno", "species_anno_match",
    "original_sbml_trans_anno", "spreadsheet_trans_anno", "converted_sbml_trans_anno", "trans_anno_match",
    "sbml_to_xlsx_time", "xlsx_to_sbml_time",
    "sbml_to_xlsx_error", "xlsx_to_sbml_error"
]
results_df = results_df[[c for c in column_order if c in results_df.columns]]

# Save to CSV
output_csv = "biodivine_260125_results.csv"
results_df.to_csv(output_csv, index=False)
print(f"Results saved to: {output_csv}")

# Summary statistics
total = len(results_df)
successful = results_df[results_df["sbml_to_xlsx_error"].isna() & results_df["xlsx_to_sbml_error"].isna()]
species_match = successful["species_match"].sum()
trans_match = successful["transitions_match"].sum()

print(f"\n=== Summary ===")
print(f"Total models: {total}")
print(f"Successfully converted: {len(successful)}")
print(f"Species count matches: {species_match}/{len(successful)}")
print(f"Transitions count matches: {trans_match}/{len(successful)}")

# Show errors if any
errors = results_df[results_df["sbml_to_xlsx_error"].notna() | results_df["xlsx_to_sbml_error"].notna()]
if len(errors) > 0:
    print(f"\nModels with errors ({len(errors)}):")
    for _, row in errors.iterrows():
        print(f"  {row['model']}: {row['sbml_to_xlsx_error'] or row['xlsx_to_sbml_error']}")

Results saved to: biodivine_260125_results.csv

=== Summary ===
Total models: 205
Successfully converted: 172
Species count matches: 172/172
Transitions count matches: 172/172

Models with errors (33):
  054_PC12-CELL-DIFFERENTIATION_source.sbml: SBML parsing errors: A <layout> object must have the required attribute 'layout:id' and may have the optional attribute 'layout:name'. No other attributes from the SBML Level 3 Layout namespace are permitted on a <layout> object.
Reference: L3V1 Layout V1 Section 3.6
 Layout attribute 'id' is missing.

  126_ORF3A_source.sbml: SBML parsing errors: An <input> object may have the optional 'metaid' and 'sboTerm' defined by SBML Level 3 Core.  No other attributes from the SBML Level 3 Core namespace or the Qualitative Models namespace are permitted on an <input> object.
Reference: L3V1 Qual V1 Section 3.6.1
  Attribute 'essential' is not part of the definition of an SBML Level 3 Version 1 Package qual Version 1 <input> element.

  078_IMMUNE-SYSTE

In [29]:
# Annotation analysis
# Note: Original SBML often uses "urn:miriam" URIs while the converter 
# produces identifiers.org URLs. The counts should still match.

print("=== Annotation Statistics ===")
print(f"\nOriginal SBML annotations:")
print(f"  Species annotations: {successful['original_sbml_species_anno'].sum()}")
print(f"  Transition annotations: {successful['original_sbml_trans_anno'].sum()}")

print(f"\nSpreadsheet annotations:")
print(f"  Species annotations: {successful['spreadsheet_species_anno'].sum()}")
print(f"  Transition annotations: {successful['spreadsheet_trans_anno'].sum()}")

print(f"\nConverted SBML annotations:")
print(f"  Species annotations: {successful['converted_sbml_species_anno'].sum()}")
print(f"  Transition annotations: {successful['converted_sbml_trans_anno'].sum()}")

# Show models with annotation differences
anno_diff = successful[
    (successful['original_sbml_species_anno'] != successful['spreadsheet_species_anno']) |
    (successful['original_sbml_trans_anno'] != successful['spreadsheet_trans_anno'])
]
if len(anno_diff) > 0:
    print(f"\nModels with annotation count differences ({len(anno_diff)}):")
    anno_cols = ["model", 
                 "original_sbml_species_anno", "spreadsheet_species_anno", "converted_sbml_species_anno",
                 "original_sbml_trans_anno", "spreadsheet_trans_anno", "converted_sbml_trans_anno"]
    display(anno_diff[anno_cols].head(20))
else:
    print("\nAll annotation counts match between original SBML and spreadsheet!")

# Timing statistics
print(f"\n=== Timing Statistics ===")
print(f"Total SBML → Spreadsheet time: {successful['sbml_to_xlsx_time'].sum():.2f}s")
print(f"Total Spreadsheet → SBML time: {successful['xlsx_to_sbml_time'].sum():.2f}s")
print(f"Average SBML → Spreadsheet time: {successful['sbml_to_xlsx_time'].mean():.3f}s")
print(f"Average Spreadsheet → SBML time: {successful['xlsx_to_sbml_time'].mean():.3f}s")
print(f"Max SBML → Spreadsheet time: {successful['sbml_to_xlsx_time'].max():.3f}s")
print(f"Max Spreadsheet → SBML time: {successful['xlsx_to_sbml_time'].max():.3f}s")


=== Annotation Statistics ===

Original SBML annotations:
  Species annotations: 34693
  Transition annotations: 2924

Spreadsheet annotations:
  Species annotations: 34736.0
  Transition annotations: 2924.0

Converted SBML annotations:
  Species annotations: 25919.0
  Transition annotations: 2649.0

Models with annotation count differences (5):


,model,original_sbml_species_anno,spreadsheet_species_anno,converted_sbml_species_anno,original_sbml_trans_anno,spreadsheet_trans_anno,converted_sbml_trans_anno
61,241_RHEUMATOID-ARTHRITIS-TH1_source.sbml,674,676.0,527.0,71,71.0,68.0
102,130_RENIN-ANGIOTENSIN_source.sbml,238,246.0,241.0,208,208.0,173.0
150,113_ER-STRESS_source.sbml,631,634.0,634.0,361,361.0,302.0
166,242_RHEUMATOID-ARTHRITIS-FIBROBLAST_source.sbml,17086,17112.0,9553.0,76,76.0,76.0
177,124_NSP9-PROTEIN_source.sbml,788,792.0,788.0,300,300.0,299.0



=== Timing Statistics ===
Total SBML → Spreadsheet time: 13.07s
Total Spreadsheet → SBML time: 20.23s
Average SBML → Spreadsheet time: 0.076s
Average Spreadsheet → SBML time: 0.118s
Max SBML → Spreadsheet time: 0.509s
Max Spreadsheet → SBML time: 0.974s


# Mismatching annotations

In [43]:
# For models with annotation count differences, extract and compare annotations

def get_sbml_species_annotations(sbml_path):
    """Extract all species annotations from SBML file as a dict of {species_id: [uri, ...]}"""
    reader = libsbml.SBMLReader()
    document = reader.readSBML(sbml_path)
    model = document.getModel()
    if model is None:
        return {}
    
    qual_plugin = model.getPlugin("qual")
    if not qual_plugin:
        return {}
    
    annotations = {}
    for i in range(qual_plugin.getNumQualitativeSpecies()):
        qs = qual_plugin.getQualitativeSpecies(i)
        species_id = qs.getId()
        if qs.isSetAnnotation():
            anno_str = qs.getAnnotationString()
            # Extract all resource URIs
            resources = re.findall(r'<rdf:li[^>]*resource="([^"]+)"', anno_str)
            annotations[species_id] = resources
        else:
            annotations[species_id] = []
    return annotations


def get_spreadsheet_species_annotations(xlsx_path):
    """Extract all species annotations from spreadsheet as a dict of {species_id: [uri, ...]}"""
    from TabularQual_converter.spreadsheet_reader import read_spreadsheet_to_model
    model, _ = read_spreadsheet_to_model(xlsx_path)
    
    annotations = {}
    for species_id, sp in model.species.items():
        # Annotations are stored as (qualifier, identifier) tuples
        annotations[species_id] = [ident for qual, ident in sp.annotations]
    return annotations


def normalize_uri(uri):
    """Normalize URI to comparable format (strip urn:miriam: prefix, convert to identifiers.org style)"""
    # # Handle urn:miriam format: urn:miriam:TYPE:ID -> TYPE:ID
    # if uri.startswith("urn:miriam:"):
    #     parts = uri[11:].split(":", 1)
    #     if len(parts) == 2:
    #         return f"{parts[0]}:{parts[1]}"
    #     return uri[11:]
    # # Handle identifiers.org format: https://identifiers.org/TYPE/ID -> TYPE:ID
    # if "identifiers.org/" in uri:
    #     match = re.search(r'identifiers\.org/([^/]+)/([^/]+)$', uri)
    #     if match:
    #         return f"{match.group(1)}:{match.group(2)}"
    # # If already in TYPE:ID form (e.g., pubmed:12345), just return as is
    # match_simple = re.match(r'^[^:/]+:[^:/]+$', uri)
    # if match_simple:
    #     return uri

    # Just remove the prefix "https://identifiers.org/" or "urn:miriam:"
    return uri.replace("https://identifiers.org/", "").replace("urn:miriam:", "")


def compare_annotations(model_file):
    base_name = os.path.splitext(model_file)[0]
    original_sbml_path = os.path.join(biodivine_model_dir, model_file)
    spreadsheet_path = os.path.join(spreadsheet_output_dir, f"{base_name}.xlsx")
    converted_sbml_path = os.path.join(sbml_output_dir, f"{base_name}_converted.sbml")
    
    # Get annotations from each source
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        orig_annos = get_sbml_species_annotations(original_sbml_path)
        xlsx_annos = get_spreadsheet_species_annotations(spreadsheet_path)
        conv_annos = get_sbml_species_annotations(converted_sbml_path)
    
    # Compare original SBML vs Spreadsheet
    print(f"\n--- SBML → Spreadsheet differences ---")
    orig_total = sum(len(v) for v in orig_annos.values())
    xlsx_total = sum(len(v) for v in xlsx_annos.values())
    print(f"Original SBML: {orig_total} annotations, Spreadsheet: {xlsx_total} annotations")
    
    for species_id in set(orig_annos.keys()) | set(xlsx_annos.keys()):
        orig_set = set(normalize_uri(u) for u in orig_annos.get(species_id, []))
        xlsx_set = set(normalize_uri(u) for u in xlsx_annos.get(species_id, []))
        
        only_in_orig = orig_set - xlsx_set
        only_in_xlsx = xlsx_set - orig_set
        
        if only_in_orig or only_in_xlsx:
            print(f"\n  Species '{species_id}':")
            if only_in_orig:
                print(f"    Only in original SBML ({len(only_in_orig)}):")
                for u in list(only_in_orig)[:10]:  # Show max 10
                    print(f"      - {u}")
                if len(only_in_orig) > 10:
                    print(f"      ... and {len(only_in_orig) - 10} more")
            if only_in_xlsx:
                print(f"    Only in spreadsheet ({len(only_in_xlsx)}):")
                for u in list(only_in_xlsx)[:10]:
                    print(f"      - {u}")
                if len(only_in_xlsx) > 10:
                    print(f"      ... and {len(only_in_xlsx) - 10} more")
    
    # Compare Spreadsheet vs Converted SBML
    print(f"\n--- Spreadsheet → SBML differences ---")
    conv_total = sum(len(v) for v in conv_annos.values())
    print(f"Spreadsheet: {xlsx_total} annotations, Converted SBML: {conv_total} annotations")
    
    diff_count = 0
    for species_id in set(xlsx_annos.keys()) | set(conv_annos.keys()):
        xlsx_set = set(normalize_uri(u) for u in xlsx_annos.get(species_id, []))
        conv_set = set(normalize_uri(u) for u in conv_annos.get(species_id, []))
        
        only_in_xlsx = xlsx_set - conv_set
        only_in_conv = conv_set - xlsx_set
        
        if only_in_xlsx or only_in_conv:
            diff_count += 1
            if diff_count <= 10:  # Show details for first 10 species only
                print(f"\n  Species '{species_id}':")
                if only_in_xlsx:
                    print(f"    Only in spreadsheet ({len(only_in_xlsx)}):")
                    for u in list(only_in_xlsx)[:10]:
                        print(f"      - {u}")
                    if len(only_in_xlsx) > 10:
                        print(f"      ... and {len(only_in_xlsx) - 10} more")
                if only_in_conv:
                    print(f"    Only in converted SBML ({len(only_in_conv)}):")
                    for u in list(only_in_conv)[:10]:
                        print(f"      - {u}")
    
    if diff_count > 10:
        print(f"\n  ... and {diff_count - 10} more species with differences")

In [44]:
compare_annotations("241_RHEUMATOID-ARTHRITIS-TH1_source.sbml")


--- SBML → Spreadsheet differences ---
Original SBML: 674 annotations, Spreadsheet: 676 annotations

  Species 'sa3388':
    Only in original SBML (1):
      - urn:casq:function:(CCL4,5/CCR5_complex|CXCL10/CXCR3_complex|AGT/F2R_complex|CCL2/CCR2_complex)&amp;GNAI/GNB/GNG_complex
    Only in spreadsheet (2):
      - urn:casq:function:(CCL4
      - 5/CCR5_complex|CXCL10/CXCR3_complex|AGT/F2R_complex|CCL2/CCR2_complex)&GNAI/GNB/GNG_complex

  Species 'sa3423':
    Only in original SBML (1):
      - urn:casq:function:NFKB1/RELA/NFKBIA_complex&amp;IKK_complex
    Only in spreadsheet (1):
      - urn:casq:function:NFKB1/RELA/NFKBIA_complex&IKK_complex

  Species 'sa3537':
    Only in original SBML (1):
      - urn:casq:function:VAV1&amp;!RHOA&amp;!SEMA4/PLXNB1_complex
    Only in spreadsheet (1):
      - urn:casq:function:VAV1&!RHOA&!SEMA4/PLXNB1_complex

  Species 'sa2925':
    Only in original SBML (1):
      - urn:casq:function:STAT4/STAT4_complex&amp;tbx21_phosphorylated&amp;RUNX3&amp;E

In [45]:
compare_annotations("130_RENIN-ANGIOTENSIN_source.sbml")


--- SBML → Spreadsheet differences ---
Original SBML: 238 annotations, Spreadsheet: 246 annotations

  Species 'sa23':
    Only in original SBML (1):
      - urn:casq:function:angiotensin I_simple_molecule&amp;ACE_cell|S|angiotensin 1-12_simple_molecule&amp;CMA1|angiotensin I_simple_molecule&amp;CMA1|angiotensin I_simple_molecule&amp;KLK1
    Only in spreadsheet (1):
      - urn:casq:function:angiotensin I_simple_molecule&ACE_cell|S|angiotensin 1-12_simple_molecule&CMA1|angiotensin I_simple_molecule&CMA1|angiotensin I_simple_molecule&KLK1

  Species 'csa4':
    Only in original SBML (1):
      - urn:casq:function:AGTR1_cell&amp;ACE2_cell&amp;!angiotensin II_simple_molecule_cell
    Only in spreadsheet (1):
      - urn:casq:function:AGTR1_cell&ACE2_cell&!angiotensin II_simple_molecule_cell

  Species 'sa50':
    Only in original SBML (1):
      - urn:casq:function:MRGPRD_cell&amp;alamandine_simple_molecule|MRGPRD_cell&amp;angiotensin 1-7_simple_molecule_cell
    Only in spreadsheet (1)